## Test Bertopic

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os
import json
from datetime import datetime
from dateutil.parser import parse
from dotenv import load_dotenv

#from ITBA-NLP.NLP_tools import Cleaning_text, top_keywords, top_entities, get_topic_name, best_document, init_opensearch, clean_all

import spacy

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
import unicodedata
from tqdm import tqdm

from umap import UMAP
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from bertopic.vectorizers import ClassTfidfTransformer

#from opensearch_data_model import Topic, TopicKeyword, News, os_client, TOPIC_INDEX_NAME, NEWS_INDEX_NAME

In [2]:
from datasets import load_dataset_builder
from datasets import load_dataset
from datasets import get_dataset_split_names

In [3]:
dataset = load_dataset("jganzabalseenka/news_2024-06-02_24hs")

In [13]:
get_dataset_split_names("jganzabalseenka/news_2024-06-02_24hs")

['train']

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['asset_id', 'title_ch', 'media', 'impact', 'start_time_utc', 'start_time_local', 'entities_curated', 'entities', 'predicted_at_entities', 'entities_raw_transformers', 'entities_transformers', 'title', 'text', 'keywords', 'predicted_at_keywords', 'truncated_text', 'title_and_text', 'prediction_delay_predictions', 'prediction_delay'],
        num_rows: 15094
    })
})

In [5]:
dataset['train']

Dataset({
    features: ['asset_id', 'title_ch', 'media', 'impact', 'start_time_utc', 'start_time_local', 'entities_curated', 'entities', 'predicted_at_entities', 'entities_raw_transformers', 'entities_transformers', 'title', 'text', 'keywords', 'predicted_at_keywords', 'truncated_text', 'title_and_text', 'prediction_delay_predictions', 'prediction_delay'],
    num_rows: 15094
})

In [6]:
df = pd.DataFrame(dataset['train'])
df.head(1)

,asset_id,title_ch,media,impact,start_time_utc,start_time_local,entities_curated,entities,predicted_at_entities,entities_raw_transformers,entities_transformers,title,text,keywords,predicted_at_keywords,truncated_text,title_and_text,prediction_delay_predictions,prediction_delay
0,110445113,Sorpresa en Atlético Nacional: se quedaría con...,Infobae,566715,2024-06-02 03:18:00,2024-06-02 00:18:00,[],"[Edwin Cardona, Juan Felipe, Juan David Londoñ...",2024-06-02 03:33:45.737059,"[{'entities': [{'end': 29, 'entity_group': 'OR...","[Atlético Nacional, Diego Ariza, Pablo Reppeto...",Sorpresa en Atlético Nacional: se quedaría con...,"2 Jun, 2024 Por Diego Ariza Nuevo Atlético Na...","[institución, periodista árabe, club, libertad...",2024-06-02 03:46:56.781300,"2 Jun, 2024 Por Diego Ariza Nuevo Atlético Na...",Sorpresa en Atlético Nacional: se quedaría con...,0.219735,0.482439


In [24]:
len(df)

15094

In [5]:
load_dotenv()
PATH_REMOTO='/content/ITBA-NLP/data/'
PATH=os.environ.get('PATH_LOCAL', PATH_REMOTO)

In [6]:
df_params = {'0_1000':'0_1000_data.parquet' }

chunk = '0_1000'

df_parquet = pd.read_parquet(PATH+df_params[chunk])

df =  df_parquet[:1000]
data_title = list(df['in__title'])
data = list(df['in__text'])
# Cantidad total de documentos
len(data)

1000

In [ ]:
data_title[:10]

## Test 1

In [90]:
tf_vectorizer = CountVectorizer(
    encoding='utf-8',
    # strip_accents='ascii',
    # lowercase=True, #(default)
    # tokenizer=None,
    # stop_words=SPANISH_STOPWORDS,
    ngram_range=(1, 2),
    # max_df=0.8,
    # min_df=0.1,
    # max_features=None,   
    # vocabulary=None,
)
tf_vectorizer.fit(data)

topic_model = BERTopic(
    language='spanish',
    #calculate_probabilities=True,
    vectorizer_model=tf_vectorizer
)

topics, probs = topic_model.fit_transform(data)

In [91]:
fn = tf_vectorizer.get_feature_names_out()
len(fn)

272121

In [92]:
T = topic_model.get_document_info(data)
docs_per_topics = T.groupby(["Topic"]).apply(lambda x: x.index).to_dict()
docs_per_topics

{-1: Index([  3,  10,  20,  22,  24,  25,  26,  28,  32,  34,
        ...
        946, 948, 955, 962, 964, 968, 974, 985, 993, 998],
       dtype='int64', length=221),
 0: Index([ 23,  39,  82,  93,  95,  99, 101, 105, 106, 119,
        ...
        952, 958, 963, 970, 975, 976, 979, 981, 988, 990],
       dtype='int64', length=166),
 1: Index([ 13,  14,  17,  29,  49,  50,  53,  89,  91, 107,
        ...
        934, 941, 943, 949, 953, 960, 967, 973, 986, 999],
       dtype='int64', length=108),
 2: Index([  4,   5,  18,  36,  41,  58,  63,  64,  70,  77,  79,  81,  85,  87,
         90,  94,  98, 115, 116, 118, 122, 125, 141, 143, 144, 147, 156, 196,
        215, 219, 235, 250, 286, 291, 308, 315, 318, 321, 323, 347, 351, 366,
        374, 384, 390, 400, 403, 437, 440, 441, 444, 451, 469, 481, 482, 485,
        496, 538, 553, 555, 560, 565, 567, 573, 605, 608, 617, 626, 669, 672,
        712, 732, 771, 776, 786, 820, 823, 848, 862, 877, 891, 913, 917, 922,
        927, 935, 951, 959,

In [94]:
topico = 0
for doc in docs_per_topics[topico]:
    print(doc," ",data_title[doc], probs[doc])
len(docs_per_topics[topico])

23   Arranca la Copa Sudamericana: la agenda con el debut de Belgrano de Córdoba, Argentinos Juniors y Defensa y Justicia 1.0
39   La sugerente frase de una ex figura de Boca Juniors antes de enfrentar al Inter Miami: "Messi es un atractivo diferente para los árbitros" 0.9797422302944493
82   Arden las tablas de la Copa de la Liga: Godoy Cruz es el primer clasificado, Boca afuera, River adentro y los posibles cruces de cuartos 0.6713453596041968
93   Comienza la ilusión continental: premios, formatos, partidos y todo lo que hay que saber de las Copas Libertadores y Sudamericana 1.0
95   La portada de un diario español que profundiza sobre el interés del Real Madrid por Franco Mastantuono: "Lo quieren ya" 1.0
99   Vivió y corrió en Inglaterra durante la Guerra de Malvinas, compitió contra Ayrton Senna y probó un Fórmula 1: la increíble historia de Enrique Mansilla 0.6861234458332404
101   ¿En qué consiste el 'Meet & Greet Upgrade' con Travis Barker y cuánto cuesta? 0.48130452289007114
1

166

In [102]:
topico = 8
for doc in docs_per_topics[topico]:
    print(doc," ",data_title[doc], probs[doc])
len(docs_per_topics[topico])

55   Caos y demoras tras el fin de semana XXL: filas kilométricas en el regreso a la Ciudad de Buenos Aires 1.0
68   Llegó el último día para una leyenda de Las Vegas: cierra el histórico casino Tropicana 0.42921704889911017
112   Durante el fin de semana XXL viajaron más de 3,2 millones de turistas: cuánto gastaron en su descanso 0.5596687660132279
139   El gobierno uruguayo celebró el regreso del turismo argentino durante la Semana Santa 0.9313543510270966
177   Con el 90% de ocupación hotelera, Termas de Río Hondo fue uno de los destinos más elegidos en Semana Santa 1.0
227   Mejores números que en el verano: cuáles fueron los destinos de Río Negro que fueron furor en Semana Santa 0.9534364974378543
266   Día de la Donación de Médula Ósea: lanzan una colecta de sangre 0.5714811750307589
290   El pueblo colonial lleno de paz que es ideal para visitar un día y escapar de la ciudad - MDZ Online 0.8975287682406622
331   Leo Nardini inauguró los pavimentos de las calles Honduras, Cuba y 

27

In [105]:
len(docs_per_topics[-1])

221

In [30]:
# Obtenemos embeddings de todos los documentos
docs_embedding = topic_model.embedding_model.embed(data)

In [31]:
len(docs_embedding)

100

In [32]:
len(docs_embedding[0])

384

- CONCLUSION: 
    - Con 100 registros:
    - sin preprocesamiento de data
    - sin vocabulario
    - sin ajustes de bertopic
    - solo CountVectorizer
        - Resultado Clasifica en dos Topicos, el resto al -1.

## Test 2 - con TFIDF

In [103]:
tfidf_vectorizer_2 = TfidfVectorizer(
    # tokenizer=tokenizer,
    # max_df=0.8,
    # min_df=0.2,
    ngram_range=(1, 2),
    #stop_words=SPANISH_STOPWORDS,
    #lowercase=False,
    #vocabulary=all_tokens,
    # max_features=100_000
)
tfidf_vectorizer_2.fit(data)

topic_model_2 = BERTopic(
    language='spanish',
    #calculate_probabilities=True,
    vectorizer_model=tfidf_vectorizer_2
)

topics_2, probs_2 = topic_model_2.fit_transform(data)

In [104]:
T_2 = topic_model_2.get_document_info(data)
docs_per_topics_2 = T_2.groupby(["Topic"]).apply(lambda x: x.index).to_dict()
docs_per_topics_2

{-1: Index([  3,   4,  22,  28,  32,  43,  44,  47,  48,  51,
        ...
        929, 937, 940, 945, 946, 955, 962, 964, 974, 998],
       dtype='int64', length=181),
 0: Index([ 23,  39,  82,  93,  95,  99, 101, 105, 106, 119,
        ...
        952, 958, 963, 970, 975, 976, 979, 981, 988, 990],
       dtype='int64', length=166),
 1: Index([  0,  10,  12,  24,  25,  26,  34,  45,  56,  59,
        ...
        938, 942, 948, 950, 961, 982, 992, 993, 995, 996],
       dtype='int64', length=132),
 2: Index([ 13,  14,  17,  29,  49,  50,  53,  89,  91, 107,
        ...
        934, 941, 943, 949, 953, 960, 967, 973, 986, 999],
       dtype='int64', length=108),
 3: Index([  5,  18,  36,  41,  58,  63,  64,  70,  77,  79,  81,  85,  87,  94,
         98, 116, 118, 122, 125, 141, 143, 144, 156, 196, 215, 219, 235, 250,
        286, 291, 315, 318, 321, 323, 347, 351, 366, 374, 384, 390, 400, 403,
        437, 440, 451, 469, 480, 481, 485, 496, 538, 553, 555, 560, 565, 567,
        573, 605

In [106]:
topico = 0
for doc in docs_per_topics_2[topico]:
    print(data_title[doc], probs_2[doc])
len(docs_per_topics_2[topico])

Arranca la Copa Sudamericana: la agenda con el debut de Belgrano de Córdoba, Argentinos Juniors y Defensa y Justicia 1.0
La sugerente frase de una ex figura de Boca Juniors antes de enfrentar al Inter Miami: "Messi es un atractivo diferente para los árbitros" 1.0
Arden las tablas de la Copa de la Liga: Godoy Cruz es el primer clasificado, Boca afuera, River adentro y los posibles cruces de cuartos 0.7489958417909681
Comienza la ilusión continental: premios, formatos, partidos y todo lo que hay que saber de las Copas Libertadores y Sudamericana 1.0
La portada de un diario español que profundiza sobre el interés del Real Madrid por Franco Mastantuono: "Lo quieren ya" 0.852123413019102
Vivió y corrió en Inglaterra durante la Guerra de Malvinas, compitió contra Ayrton Senna y probó un Fórmula 1: la increíble historia de Enrique Mansilla 0.6694542676485707
¿En qué consiste el 'Meet & Greet Upgrade' con Travis Barker y cuánto cuesta? 0.4303482183583134
La dramática confesión de una figura de

166

In [107]:
topico = 1
for doc in docs_per_topics_2[topico]:
    print(data_title[doc], probs_2[doc])
len(docs_per_topics_2[topico])

Elecciones en Venezuela: María Corina Machado pidió más apoyo de Noruega para garantizar unas presidenciales libres 1.0
En abril crecerá la protesta sindical: se prevé una oleada de paros que incluirá al transporte, estatales, docentes y la CGT 0.7364586229850156
Trias no descarta un pacto Junts-PSC tras el 12-M: "El problema es para hacer qué" 0.9237910520248518
El PP acusa a Sánchez y Begoña Gómez de "tráfico de influencias conyugal" y Bolaños dice que sus palabras son miserables 0.9168429875859485
El Gobierno investiga más irregularidades: el software de casi un millón de dólares y los vehículos de alta gama 0.7818554361019956
Estados Unidos y el rol de la APEP 0.7086859123836917
Se profundiza la crisis inmobiliaria de China: Country Garden suspendió la cotización en la bolsa de Hong Kong 0.964301605663641
NC-BC critica la "inacción" del Gobierno con las maniobras militares de Marruecos 1.0
La CTA Autónoma alertó que habrá nuevas medidas por los despidos en el Estado tras las protes

132

In [109]:
len(docs_per_topics_2[-1])

181

- 2. Con 100 registros:
    - sin preprocesamiento
    - sin vocabulario
    - sin ajustes de bertopic
    - con TFIDFVectoizer
        - Resultado: Clasifica en dos topicos tambien, pero restringe un poco la cantidad de docs:
            - del topico 0, de 71 (Test 1) a 61.
            - del topico 1, de 21 (Test 1) a 19.
            - Esto podria explicar que selecciona mejor los docs del topico 0, que es usualmente el de mayor cantidad de docs.

## Test 3 - con TFIDF y control de parametros y stopwords

In [110]:
# Stopwords
SPANISH_STOPWORDS = list(pd.read_csv(PATH+'spanish_stop_words.csv' )['stopwords'].values)
SPANISH_STOPWORDS_SIN_ACENTOS = [unicodedata.normalize('NFKD', word).encode('ASCII', 'ignore').decode('ASCII') for word in SPANISH_STOPWORDS] 

tfidf_vectorizer_3 = TfidfVectorizer(
    # tokenizer=tokenizer,
    max_df=0.8,
    min_df=2,
    ngram_range=(1, 2),
    stop_words=SPANISH_STOPWORDS,
    #lowercase=False,
    #vocabulary=all_tokens,
    # max_features=100_000
)
tfidf_vectorizer_3.fit(data)

topic_model_3 = BERTopic(
    language='spanish',
    #calculate_probabilities=True,
    vectorizer_model=tfidf_vectorizer_3
)

topics_3, probs_3 = topic_model_3.fit_transform(data)

In [111]:
len(tfidf_vectorizer_3.get_feature_names_out())

23995

In [112]:
T_3 = topic_model_3.get_document_info(data)
docs_per_topics_3 = T_3.groupby(["Topic"]).apply(lambda x: x.index).to_dict()
docs_per_topics_3

{-1: Index([  3,  10,  22,  28,  29,  32,  43,  44,  46,  47,
        ...
        940, 945, 946, 955, 962, 964, 968, 974, 985, 998],
       dtype='int64', length=189),
 0: Index([ 23,  39,  82,  93,  95,  99, 101, 105, 106, 119,
        ...
        952, 958, 963, 970, 975, 976, 979, 981, 988, 990],
       dtype='int64', length=166),
 1: Index([  0,  12,  24,  25,  26,  34,  45,  56,  59,  65,
        ...
        938, 942, 948, 950, 961, 982, 992, 993, 995, 996],
       dtype='int64', length=134),
 2: Index([ 13,  14,  17,  49,  50,  53,  89,  91, 107, 109,
        ...
        934, 941, 943, 949, 953, 960, 967, 973, 986, 999],
       dtype='int64', length=108),
 3: Index([  4,   5,  18,  36,  41,  58,  63,  64,  70,  77,  79,  81,  85,  87,
         94,  98, 115, 116, 118, 122, 125, 141, 143, 144, 153, 156, 196, 215,
        219, 235, 250, 286, 291, 299, 308, 315, 318, 321, 323, 347, 351, 366,
        374, 384, 390, 400, 403, 437, 440, 441, 451, 469, 481, 482, 485, 496,
        538, 553

In [113]:
topico = 0
for doc in docs_per_topics_3[topico]:
    print(data_title[doc], probs_3[doc])
len(docs_per_topics_3[topico])

Arranca la Copa Sudamericana: la agenda con el debut de Belgrano de Córdoba, Argentinos Juniors y Defensa y Justicia 1.0
La sugerente frase de una ex figura de Boca Juniors antes de enfrentar al Inter Miami: "Messi es un atractivo diferente para los árbitros" 1.0
Arden las tablas de la Copa de la Liga: Godoy Cruz es el primer clasificado, Boca afuera, River adentro y los posibles cruces de cuartos 0.8354110274714052
Comienza la ilusión continental: premios, formatos, partidos y todo lo que hay que saber de las Copas Libertadores y Sudamericana 1.0
La portada de un diario español que profundiza sobre el interés del Real Madrid por Franco Mastantuono: "Lo quieren ya" 0.9758871228241844
Vivió y corrió en Inglaterra durante la Guerra de Malvinas, compitió contra Ayrton Senna y probó un Fórmula 1: la increíble historia de Enrique Mansilla 0.6860807699699709
¿En qué consiste el 'Meet & Greet Upgrade' con Travis Barker y cuánto cuesta? 0.5745292420833042
La dramática confesión de una figura d

166

In [114]:
topico = 1
for doc in docs_per_topics_3[topico]:
    print(data_title[doc], probs_3[doc])
len(docs_per_topics_3[topico])

Elecciones en Venezuela: María Corina Machado pidió más apoyo de Noruega para garantizar unas presidenciales libres 1.0
Trias no descarta un pacto Junts-PSC tras el 12-M: "El problema es para hacer qué" 1.0
El PP acusa a Sánchez y Begoña Gómez de "tráfico de influencias conyugal" y Bolaños dice que sus palabras son miserables 0.9408213071943045
El Gobierno investiga más irregularidades: el software de casi un millón de dólares y los vehículos de alta gama 0.8517837742994738
Estados Unidos y el rol de la APEP 0.8699360437332312
Se profundiza la crisis inmobiliaria de China: Country Garden suspendió la cotización en la bolsa de Hong Kong 0.9925774166493564
NC-BC critica la "inacción" del Gobierno con las maniobras militares de Marruecos 1.0
La CTA Autónoma alertó que habrá nuevas medidas por los despidos en el Estado tras las protestas de mañana 0.972890633774001
Oppenheimer habló tras la entrevista a Milei: "Confía en que la situación va mejorar antes de que se agote la paciencia social

134

In [115]:
len(docs_per_topics_3[-1])

189

- 3. Con 100 registros:
    - sin preprocesamiento
    - sin vocabulario
    - sin ajustes de bertopic
    - con TFIDFVectoizer + parametrizacion min_df max_df y stop words
        - Resultado: Clasifica en dos topicos tambien, se obtiene un resultado similar al Test 2
            - del topico 0, de 61 (Test 2) a 65.
            - del topico 1, de 19 (Test 2) a 18.
            

## Test 4 - Entrenamos agregando un vocabulario con stopwords
- Esto implica que el TfidfVectorizer solo considerará las palabras y n-gramas que están en ese vocabulario, no realizará ninguna acción sobre las palabras de los documentos.


In [116]:
# Cargar el modelo de spaCy para español
spa = spacy.load("es_core_news_lg")

# Stopwords
SPANISH_STOPWORDS = list(pd.read_csv(PATH+'spanish_stop_words.csv' )['stopwords'].values)
SPANISH_STOPWORDS_SIN_ACENTOS = [unicodedata.normalize('NFKD', word).encode('ASCII', 'ignore').decode('ASCII') for word in SPANISH_STOPWORDS] 

# Detectar entidades para todos los documentos usando spaCy
entities_spa = []
for doc in tqdm(data):
    extract = spa(doc)
    entities_spa.append([(ent.text, ent.label_) for ent in extract.ents])

# Procesamiento de entidades encontradas
entities = []
original_entities = []
word_count = {}
for item in tqdm(entities_spa):
    for ent in item:
        if ent[1] == 'PER' or ent[1] == 'ORG' or ent[1] == 'LOC':
            token = str(ent[0]).lower()  
            if token not in word_count:
                word_count[token] = {'count': 0, 'original': ent[0]}
                
            word_count[token]['count'] += 1   
    
    # Ordenar el diccionario por el valor del conteo en orden descendente
    sorted_word_count = dict(sorted(word_count.items(), key=lambda item: item[1]['count'], reverse=True))  

    word_count = {}

    # Crear la lista de entidades procesadas por noticia (para guardar en DB)
    original_entities.append( [value['original'] for _, value in sorted_word_count.items() 
                                if value['original'].lower() not in SPANISH_STOPWORDS_SIN_ACENTOS and len(value['original'].split()) < 5] )

    # Crear la lista de entidades procesadas por noticia (entrenamiento)
    pre_entities = clean_all( [key for key, _ in sorted_word_count.items()
                                if key not in SPANISH_STOPWORDS_SIN_ACENTOS and len(key.split()) < 5 ] )

    # entidades para entrenar
    entities.append( pre_entities )


100%|██████████| 1000/1000 [00:00<00:00, 1174.97it/s]


In [117]:
# Vocabulario
preprocessed_entities = []
registro = ""
for array in entities:
    
    for ent in array:
        if len(ent.split()) > 2:
            for word in ent.split():
                if word.isalpha() and len(word) > 2 and len(word) < 15 and word not in SPANISH_STOPWORDS_SIN_ACENTOS:    
                    preprocessed_entities.append(word)
                elif word.isnumeric() and len(word) == 4:
                    preprocessed_entities.append(word)
        else:
            for word in ent.split():
                if word.isalpha() and len(word) > 2 and len(word) < 15 and word not in SPANISH_STOPWORDS_SIN_ACENTOS:  
                    registro += word + " "
            if registro:
                preprocessed_entities.append(registro.strip())
                registro = ""

vocab_entities = list(set([item for item in preprocessed_entities]))
len(vocab_entities)

9272

In [118]:
tfidf_vectorizer_4 = TfidfVectorizer(
    # tokenizer=tokenizer,
    # max_df=0.8,
    # min_df=0.2,
    ngram_range=(1, 2),
    #stop_words=SPANISH_STOPWORDS,
    lowercase=True,
    vocabulary=vocab_entities,
    # max_features=100_000
)
tfidf_vectorizer_4.fit(data)

topic_model_4 = BERTopic(
    language='spanish',
    #calculate_probabilities=True,
    vectorizer_model=tfidf_vectorizer_4
)

topics_4, probs_4 = topic_model_4.fit_transform(data)

In [119]:
T_4 = topic_model_4.get_document_info(data)
docs_per_topics_4 = T_4.groupby(["Topic"]).apply(lambda x: x.index).to_dict()
docs_per_topics_4

{-1: Index([  3,   4,  10,  12,  21,  22,  24,  25,  26,  43,
        ...
        938, 940, 945, 946, 948, 955, 962, 974, 985, 993],
       dtype='int64', length=191),
 0: Index([ 23,  39,  82,  93,  95,  99, 101, 105, 106, 119,
        ...
        952, 958, 963, 970, 975, 976, 979, 981, 988, 990],
       dtype='int64', length=167),
 1: Index([  1,   6,   9,  15,  20,  30,  35,  40,  46,  47,
        ...
        889, 902, 909, 910, 912, 932, 964, 969, 997, 998],
       dtype='int64', length=114),
 2: Index([ 13,  14,  17,  29,  49,  50,  53,  89,  91, 107,
        ...
        934, 941, 943, 949, 953, 960, 967, 973, 986, 999],
       dtype='int64', length=106),
 3: Index([  5,  18,  36,  41,  58,  63,  64,  70,  77,  79,  81,  85,  87,  94,
         98, 115, 116, 118, 122, 125, 141, 143, 144, 147, 153, 156, 196, 215,
        219, 235, 249, 250, 286, 291, 299, 315, 318, 321, 323, 347, 351, 366,
        374, 384, 390, 400, 403, 437, 440, 441, 451, 469, 481, 482, 485, 496,
        538, 553

In [120]:
topico = 0
for doc in docs_per_topics_4[topico]:
    print(data_title[doc], probs_4[doc])
len(docs_per_topics_4[topico])

Arranca la Copa Sudamericana: la agenda con el debut de Belgrano de Córdoba, Argentinos Juniors y Defensa y Justicia 1.0
La sugerente frase de una ex figura de Boca Juniors antes de enfrentar al Inter Miami: "Messi es un atractivo diferente para los árbitros" 1.0
Arden las tablas de la Copa de la Liga: Godoy Cruz es el primer clasificado, Boca afuera, River adentro y los posibles cruces de cuartos 0.8002622142845542
Comienza la ilusión continental: premios, formatos, partidos y todo lo que hay que saber de las Copas Libertadores y Sudamericana 1.0
La portada de un diario español que profundiza sobre el interés del Real Madrid por Franco Mastantuono: "Lo quieren ya" 0.9359658730815517
Vivió y corrió en Inglaterra durante la Guerra de Malvinas, compitió contra Ayrton Senna y probó un Fórmula 1: la increíble historia de Enrique Mansilla 0.6595244515554267
¿En qué consiste el 'Meet & Greet Upgrade' con Travis Barker y cuánto cuesta? 0.4930727414865186
La dramática confesión de una figura d

167

In [121]:
topico = 1
for doc in docs_per_topics_4[topico]:
    print(data_title[doc], probs_4[doc])
len(docs_per_topics_4[topico])

El populismo derivó en que los argentinos financien el crecimiento de los países desarrollados 0.7483941548342843
Dólar versus inflación: ¿cuál debería ser el nivel del tipo de cambio? 1.0
Elogios de un profesor argentino que enseña en Harvard: "Para hacer progresismo hay que hacer lo que está haciendo Milei" 0.8504157102733593
Bitcoin: cómo se ha movido en el mercado este 2 de abril 0.7194427483318
Un teclado puede albergar más gérmenes que un inodoro: qué tipo de bacterías 0.5370269800431061
Criptomonedas: el precio de ethereum para este día 0.6568909052531329
Se disparó el interés por alquilar departamentos de 4 ambientes en CABA: la oferta subió más de 80 por ciento 0.890093339127818
Criptomonedas: el valor de tether para este 2 de abril 0.7194427483318
Exclusivo: con quiénes está negociando el Gobierno para conseguir los USD 15.000 millones que necesita para levantar el cepo 0.5635723857482926
Café en el horno microondas: Tips para calentarlo de manera segura y sin comprometer el 

114

In [122]:
len(docs_per_topics_4[-1])

191

- 4. Con 100 registros:
    - sin preprocesamiento 
    - con vocabulario
    - sin ajustes de bertopic
    - con TFIDFVectorizer que aplica al vocabulario
        - Resultado: Clasifica en dos topicos tambien, se obtiene un resultado similar al Test 3
            - del topico 0, de 65 (Test 3) a 69.
            - del topico 1, de 18 (Test 3) a 21.

Cabe mencionar que la cantidad de palabras del vocabulario es de 2012 vs 2279 que se obtienen de TFIDF sobre el total de los documentos

## Test 5- solo con palabras del vocabulario que se repiten mas de una vez

In [57]:
# Cargar el modelo de spaCy para español
spa = spacy.load("es_core_news_lg")

# Stopwords
SPANISH_STOPWORDS = list(pd.read_csv(PATH+'spanish_stop_words.csv' )['stopwords'].values)
SPANISH_STOPWORDS_SIN_ACENTOS = [unicodedata.normalize('NFKD', word).encode('ASCII', 'ignore').decode('ASCII') for word in SPANISH_STOPWORDS] 

# Detectar entidades para todos los documentos usando spaCy
entities_spa = []
for doc in tqdm(data):
    extract = spa(doc)
    entities_spa.append([(ent.text, ent.label_) for ent in extract.ents])

# Procesamiento de entidades encontradas
entities = []
original_entities = []
word_count = {}
for item in tqdm(entities_spa):
    for ent in item:
        if ent[1] == 'PER' or ent[1] == 'ORG' or ent[1] == 'LOC':
            token = str(ent[0]).lower()  
            if token not in word_count:
                word_count[token] = {'count': 0, 'original': ent[0]}
                
            word_count[token]['count'] += 1   
    
    # Ordenar el diccionario por el valor del conteo en orden descendente
    sorted_word_count = dict(sorted(word_count.items(), key=lambda item: item[1]['count'], reverse=True))  

    word_count = {}

    # Crear la lista de entidades procesadas por noticia (para guardar en DB)
    original_entities.append( [value['original'] for _, value in sorted_word_count.items() 
                                if value['original'].lower() not in SPANISH_STOPWORDS_SIN_ACENTOS and len(value['original'].split()) < 5] )

    # Crear la lista de entidades procesadas por noticia (entrenamiento)
    pre_entities = clean_all( [key for key, _ in sorted_word_count.items()
                                if key not in SPANISH_STOPWORDS_SIN_ACENTOS and len(key.split()) < 5 and _['count'] > 1 ] )

    # entidades para entrenar
    entities.append( pre_entities )

100%|██████████| 1000/1000 [00:00<00:00, 1198.84it/s]


In [59]:
# Vocabulario
preprocessed_entities = []
registro = ""
for array in entities:
    for ent in array:
        if len(ent.split()) > 2:
            for word in ent.split():
                if word.isalpha() and len(word) > 2 and len(word) < 15 and word not in SPANISH_STOPWORDS_SIN_ACENTOS:    
                    preprocessed_entities.append(word)
                elif word.isnumeric() and len(word) == 4:
                    preprocessed_entities.append(word)
        else:
            for word in ent.split():
                if word.isalpha() and len(word) > 2 and len(word) < 15 and word not in SPANISH_STOPWORDS_SIN_ACENTOS:  
                    registro += word + " "
            if registro:
                preprocessed_entities.append(registro.strip())
                registro = ""

vocab_entities = list(set([item for item in preprocessed_entities]))
len(vocab_entities)

2912

In [65]:
tfidf_vectorizer_5 = TfidfVectorizer(
    # tokenizer=tokenizer,
    max_df=0.9,
    min_df=0.1,
    ngram_range=(1, 2),
    #stop_words=SPANISH_STOPWORDS,
    lowercase=True,
    vocabulary=vocab_entities,
    # max_features=100_000
)
tfidf_vectorizer_5.fit(data)

topic_model_5 = BERTopic(
    language='spanish',
    #calculate_probabilities=True,
    vectorizer_model=tfidf_vectorizer_5
)

topics_5, probs_5 = topic_model_5.fit_transform(data)

In [66]:
T_5 = topic_model_5.get_document_info(data)
docs_per_topics_5 = T_5.groupby(["Topic"]).apply(lambda x: x.index).to_dict()
docs_per_topics_5

{-1: Index([  3,   4,  32,  43,  44,  47,  48,  51,  66,  71,
        ...
        929, 940, 945, 946, 948, 962, 964, 968, 974, 998],
       dtype='int64', length=146),
 0: Index([ 23,  39,  82,  93,  95,  99, 101, 105, 106, 119,
        ...
        952, 958, 963, 970, 975, 976, 979, 981, 988, 990],
       dtype='int64', length=164),
 1: Index([  0,  10,  12,  24,  25,  26,  34,  45,  56,  59,
        ...
        926, 938, 942, 950, 961, 982, 992, 993, 995, 996],
       dtype='int64', length=137),
 2: Index([ 13,  14,  17,  29,  49,  50,  53,  57,  89,  91,
        ...
        941, 943, 949, 953, 955, 960, 967, 973, 986, 999],
       dtype='int64', length=120),
 3: Index([  5,  18,  36,  41,  58,  63,  64,  70,  77,  79,  81,  85,  87,  90,
         94,  98, 115, 116, 118, 122, 125, 141, 143, 144, 147, 156, 196, 215,
        219, 235, 250, 286, 291, 315, 318, 321, 323, 347, 351, 366, 374, 384,
        390, 400, 403, 437, 440, 441, 451, 469, 480, 481, 482, 485, 496, 538,
        553, 555

In [11]:
topico = 0
for doc in docs_per_topics_5[topico]:
    print(doc," ",data_title[doc], probs_5[doc])
len(docs_per_topics_5[topico])

1   El populismo derivó en que los argentinos financien el crecimiento de los países desarrollados 1.0
3   La revolución del efectivo: la Generación Z redescubre el dinero antiguo para evitar la bancarrota moderna 0.7758301051426786
6   Dólar versus inflación: ¿cuál debería ser el nivel del tipo de cambio? 1.0
7   La lectura en voz alta: clave para el desarrollo infantil en el hogar y la escuela 1.0
9   Elogios de un profesor argentino que enseña en Harvard: "Para hacer progresismo hay que hacer lo que está haciendo Milei" 1.0
11   WhatsApp habilitó más funciones nuevas en abril, conoce la lista 1.0
15   Bitcoin: cómo se ha movido en el mercado este 2 de abril 1.0
16   Google dice adiós a una de sus mejores aplicaciones después de seis años de vida 1.0
20   Un teclado puede albergar más gérmenes que un inodoro: qué tipo de bacterías 0.9463052421768003
30   Criptomonedas: el precio de ethereum para este día 1.0
35   Se disparó el interés por alquilar departamentos de 4 ambientes en CABA

184

In [12]:
topico = 1
for doc in docs_per_topics_5[topico]:
    print(doc," ",data_title[doc], probs_5[doc])
len(docs_per_topics_5[topico])

23   Arranca la Copa Sudamericana: la agenda con el debut de Belgrano de Córdoba, Argentinos Juniors y Defensa y Justicia 1.0
39   La sugerente frase de una ex figura de Boca Juniors antes de enfrentar al Inter Miami: "Messi es un atractivo diferente para los árbitros" 0.9661419491708247
82   Arden las tablas de la Copa de la Liga: Godoy Cruz es el primer clasificado, Boca afuera, River adentro y los posibles cruces de cuartos 0.7946129226210314
93   Comienza la ilusión continental: premios, formatos, partidos y todo lo que hay que saber de las Copas Libertadores y Sudamericana 1.0
95   La portada de un diario español que profundiza sobre el interés del Real Madrid por Franco Mastantuono: "Lo quieren ya" 0.9679645494820338
99   Vivió y corrió en Inglaterra durante la Guerra de Malvinas, compitió contra Ayrton Senna y probó un Fórmula 1: la increíble historia de Enrique Mansilla 0.6076370799111283
101   ¿En qué consiste el 'Meet & Greet Upgrade' con Travis Barker y cuánto cuesta? 0.4318

159

In [67]:
len(docs_per_topics_5[-1])

146

- 5. Con 100 registros:
    - sin preprocesamiento 
    - con vocabulario + repeticion de palabras: mas de una vez
    - sin ajustes de bertopic
    - con TFIDFVectorizer que aplica al vocabulario
        - Resultado: Clasifica en dos topicos tambien, se obtiene un resultado similar al Test 4
            - del topico 0, de 69 (Test 4) a 50.
            - del topico 1, de 21 (Test 4) a 16.

Cabe mencionar que la cantidad de palabras del vocabulario es de 543 vs 2279 que se obtienen de TFIDF sobre el total de los documentos

## Test 6- solo con palabras del vocabulario que se repiten mas de dos veces

In [14]:
# Cargar el modelo de spaCy para español
spa = spacy.load("es_core_news_lg")

# Stopwords
SPANISH_STOPWORDS = list(pd.read_csv(PATH+'spanish_stop_words.csv' )['stopwords'].values)
SPANISH_STOPWORDS_SIN_ACENTOS = [unicodedata.normalize('NFKD', word).encode('ASCII', 'ignore').decode('ASCII') for word in SPANISH_STOPWORDS] 

# Detectar entidades para todos los documentos usando spaCy
entities_spa = []
for doc in tqdm(data):
    extract = spa(doc)
    entities_spa.append([(ent.text, ent.label_) for ent in extract.ents])

# Procesamiento de entidades encontradas
entities = []
original_entities = []
word_count = {}
for item in tqdm(entities_spa):
    for ent in item:
        if ent[1] == 'PER' or ent[1] == 'ORG' or ent[1] == 'LOC':
            token = str(ent[0]).lower()  
            if token not in word_count:
                word_count[token] = {'count': 0, 'original': ent[0]}
                
            word_count[token]['count'] += 1   
    
    # Ordenar el diccionario por el valor del conteo en orden descendente
    sorted_word_count = dict(sorted(word_count.items(), key=lambda item: item[1]['count'], reverse=True))  

    word_count = {}

    # Crear la lista de entidades procesadas por noticia (para guardar en DB)
    original_entities.append( [value['original'] for _, value in sorted_word_count.items() 
                                if value['original'].lower() not in SPANISH_STOPWORDS_SIN_ACENTOS and len(value['original'].split()) < 5] )

    # Crear la lista de entidades procesadas por noticia (entrenamiento)
    pre_entities = clean_all( [key for key, _ in sorted_word_count.items()
                                if key not in SPANISH_STOPWORDS_SIN_ACENTOS and len(key.split()) < 5 and _['count'] > 2 ] )

    # entidades para entrenar
    entities.append( pre_entities )

100%|██████████| 1000/1000 [00:00<00:00, 2772.46it/s]


In [15]:
# Vocabulario
preprocessed_entities = []
registro = ""
for array in entities:
    for ent in array:
        if len(ent.split()) > 2:
            for word in ent.split():
                if word.isalpha() and len(word) > 2 and len(word) < 15 and word not in SPANISH_STOPWORDS_SIN_ACENTOS:    
                    preprocessed_entities.append(word)
                elif word.isnumeric() and len(word) == 4:
                    preprocessed_entities.append(word)
        else:
            for word in ent.split():
                if word.isalpha() and len(word) > 2 and len(word) < 15 and word not in SPANISH_STOPWORDS_SIN_ACENTOS:  
                    registro += word + " "
            if registro:
                preprocessed_entities.append(registro.strip())
                registro = ""

vocab_entities = list(set([item for item in preprocessed_entities]))
len(vocab_entities)

1320

In [16]:
tfidf_vectorizer_6 = TfidfVectorizer(
    # tokenizer=tokenizer,
    # max_df=0.8,
    # min_df=0.2,
    ngram_range=(1, 2),
    #stop_words=SPANISH_STOPWORDS,
    lowercase=True,
    vocabulary=vocab_entities,
    # max_features=100_000
)
tfidf_vectorizer_6.fit(data)

topic_model_6 = BERTopic(
    language='spanish',
    #calculate_probabilities=True,
    vectorizer_model=tfidf_vectorizer_6
)

topics_6, probs_6 = topic_model_6.fit_transform(data)

In [17]:
T_6 = topic_model_6.get_document_info(data)
docs_per_topics_6 = T_6.groupby(["Topic"]).apply(lambda x: x.index).to_dict()
docs_per_topics_6

{-1: Index([  3,   4,  10,  20,  21,  22,  24,  25,  26,  28,
        ...
        945, 946, 948, 955, 962, 964, 974, 982, 993, 998],
       dtype='int64', length=245),
 0: Index([ 23,  39,  82,  93,  95,  99, 101, 105, 106, 119,
        ...
        952, 958, 963, 970, 975, 976, 979, 981, 988, 990],
       dtype='int64', length=163),
 1: Index([ 13,  14,  17,  29,  49,  50,  53,  89,  91, 107,
        ...
        934, 941, 943, 949, 953, 960, 967, 973, 986, 999],
       dtype='int64', length=110),
 2: Index([  5,  18,  36,  41,  58,  63,  64,  66,  70,  77,  79,  81,  85,  87,
         90,  94,  98, 113, 115, 116, 118, 122, 125, 141, 143, 144, 147, 156,
        196, 215, 219, 235, 250, 286, 291, 299, 315, 318, 321, 323, 347, 351,
        366, 374, 384, 390, 400, 403, 437, 440, 441, 451, 469, 481, 485, 496,
        538, 553, 555, 560, 565, 567, 573, 605, 608, 617, 626, 669, 672, 712,
        732, 741, 771, 776, 786, 820, 823, 829, 848, 862, 891, 913, 917, 922,
        927, 935, 951, 959,

In [18]:
topico = 0
for doc in docs_per_topics_6[topico]:
    print(doc," ",data_title[doc], probs_6[doc])
len(docs_per_topics_6[topico])

23   Arranca la Copa Sudamericana: la agenda con el debut de Belgrano de Córdoba, Argentinos Juniors y Defensa y Justicia 1.0
39   La sugerente frase de una ex figura de Boca Juniors antes de enfrentar al Inter Miami: "Messi es un atractivo diferente para los árbitros" 1.0
82   Arden las tablas de la Copa de la Liga: Godoy Cruz es el primer clasificado, Boca afuera, River adentro y los posibles cruces de cuartos 0.756463433014935
93   Comienza la ilusión continental: premios, formatos, partidos y todo lo que hay que saber de las Copas Libertadores y Sudamericana 1.0
95   La portada de un diario español que profundiza sobre el interés del Real Madrid por Franco Mastantuono: "Lo quieren ya" 0.759026333039102
99   Vivió y corrió en Inglaterra durante la Guerra de Malvinas, compitió contra Ayrton Senna y probó un Fórmula 1: la increíble historia de Enrique Mansilla 0.5267177672842149
101   ¿En qué consiste el 'Meet & Greet Upgrade' con Travis Barker y cuánto cuesta? 0.4062603559164919
105 

163

In [19]:
topico = 1
for doc in docs_per_topics_6[topico]:
    print(doc," ",data_title[doc], probs_6[doc])
len(docs_per_topics_6[topico])

13   Un niño de 12 años mató a un compañero de clase e hirió a otros dos con un arma de fuego en Finlandia 1.0
14   Accidente en la General Paz: un auto terminó incrustado debajo de un camión 1.0
17   Una joven de 19 años fue baleada en la puerta de su casa por dos hombres que tendrían problemas con su novio 1.0
29   Dead Kennedys acusó a seguidores del primer ministro húngaro de plagiar su icónica foto 1.0
49   Se fugó de un penal y lo atraparon los vecinos cuando quiso robar en una casa de Los Hornos: lo golpearon y desnudaron 1.0
50   Fin de semana sangriento en Ecuador: 137 homicidios durante el asueto de Semana Santa 1.0
53   Un militar venezolano fue obligado a declarar que había planificado un magnicidio con apoyo de María Corina Machado 0.8940685835354047
89   El video que muestra a los jugadores de Vélez salir de la habitación del hotel tras el presunto abuso sexual 0.93585690799057
91   El régimen de Nicolás Maduro excarceló al youtuber acusado de terrorismo en Venezuela 0.93

110

In [20]:
len(docs_per_topics_6[-1])

245

- 6. Con 100 registros:
    - sin preprocesamiento 
    - con vocabulario + repeticion de palabras: mas de dos veces
    - sin ajustes de bertopic
    - con TFIDFVectorizer que aplica al vocabulario
        - Resultado: Clasifica en dos topicos tambien, se obtiene un resultado similar al Test 3
            - del topico 0, de 65 (Test 3) a 64.
            - del topico 1, de 18 (Test 3) a 21.

Cabe mencionar que la cantidad de palabras del vocabulario es de 259 vs 2279 que se obtienen de TFIDF sobre el total de los documentos

## Test 7- Palabras de vocab que se repite mas de una vez + configuracion en max_df y min_df

In [22]:
# Cargar el modelo de spaCy para español
spa = spacy.load("es_core_news_lg")

# Stopwords
SPANISH_STOPWORDS = list(pd.read_csv(PATH+'spanish_stop_words.csv' )['stopwords'].values)
SPANISH_STOPWORDS_SIN_ACENTOS = [unicodedata.normalize('NFKD', word).encode('ASCII', 'ignore').decode('ASCII') for word in SPANISH_STOPWORDS] 

# Detectar entidades para todos los documentos usando spaCy
entities_spa = []
for doc in tqdm(data):
    extract = spa(doc)
    entities_spa.append([(ent.text, ent.label_) for ent in extract.ents])

# Procesamiento de entidades encontradas
entities = []
original_entities = []
word_count = {}
for item in tqdm(entities_spa):
    for ent in item:
        if ent[1] == 'PER' or ent[1] == 'ORG' or ent[1] == 'LOC':
            token = str(ent[0]).lower()  
            if token not in word_count:
                word_count[token] = {'count': 0, 'original': ent[0]}
                
            word_count[token]['count'] += 1   
    
    # Ordenar el diccionario por el valor del conteo en orden descendente
    sorted_word_count = dict(sorted(word_count.items(), key=lambda item: item[1]['count'], reverse=True))  

    word_count = {}

    # Crear la lista de entidades procesadas por noticia (para guardar en DB)
    original_entities.append( [value['original'] for _, value in sorted_word_count.items() 
                                if value['original'].lower() not in SPANISH_STOPWORDS_SIN_ACENTOS and len(value['original'].split()) < 5] )

    # Crear la lista de entidades procesadas por noticia (entrenamiento)
    pre_entities = clean_all( [key for key, _ in sorted_word_count.items()
                                if key not in SPANISH_STOPWORDS_SIN_ACENTOS and len(key.split()) < 5 and _['count'] > 1 ] )

    # entidades para entrenar
    entities.append( pre_entities )

100%|██████████| 1000/1000 [00:00<00:00, 1704.12it/s]


In [23]:
# Vocabulario
preprocessed_entities = []
registro = ""
for array in entities:
    for ent in array:
        if len(ent.split()) > 2:
            for word in ent.split():
                if word.isalpha() and len(word) > 2 and len(word) < 15 and word not in SPANISH_STOPWORDS_SIN_ACENTOS:    
                    preprocessed_entities.append(word)
                elif word.isnumeric() and len(word) == 4:
                    preprocessed_entities.append(word)
        else:
            for word in ent.split():
                if word.isalpha() and len(word) > 2 and len(word) < 15 and word not in SPANISH_STOPWORDS_SIN_ACENTOS:  
                    registro += word + " "
            if registro:
                preprocessed_entities.append(registro.strip())
                registro = ""

vocab_entities = list(set([item for item in preprocessed_entities]))
len(vocab_entities)

2912

In [27]:
tfidf_vectorizer_7 = TfidfVectorizer(
    # tokenizer=tokenizer,
    max_df=0.9,
    min_df=0.05,
    ngram_range=(1, 2),
    #stop_words=SPANISH_STOPWORDS,
    lowercase=True,
    vocabulary=vocab_entities,
    # max_features=100_000
)
tfidf_vectorizer_7.fit(data)

topic_model_7 = BERTopic(
    language='spanish',
    #calculate_probabilities=True,
    vectorizer_model=tfidf_vectorizer_7
)

topics_7, probs_7 = topic_model_7.fit_transform(data)

In [40]:
T_7 = topic_model_7.get_document_info(data)
docs_per_topics_7 = T_7.groupby(["Topic"]).apply(lambda x: x.index).to_dict()
docs_per_topics_7

{-1: Index([  3,   4,  10,  12,  21,  24,  25,  26,  32,  34,
        ...
        946, 948, 961, 962, 964, 974, 982, 985, 993, 998],
       dtype='int64', length=234),
 0: Index([ 23,  39,  82,  93,  95,  99, 101, 105, 106, 119,
        ...
        952, 958, 963, 970, 975, 976, 979, 981, 988, 990],
       dtype='int64', length=165),
 1: Index([ 13,  14,  17,  29,  49,  50,  53,  89,  91, 107,
        ...
        941, 943, 949, 953, 955, 960, 967, 973, 986, 999],
       dtype='int64', length=112),
 2: Index([  5,  18,  36,  41,  58,  63,  64,  70,  77,  79,  81,  87,  94,  98,
        115, 116, 118, 122, 125, 141, 143, 144, 147, 156, 196, 215, 219, 235,
        250, 286, 291, 299, 308, 315, 318, 321, 323, 347, 351, 366, 374, 384,
        390, 400, 403, 437, 440, 441, 451, 469, 481, 482, 485, 496, 538, 553,
        555, 560, 565, 567, 573, 605, 608, 617, 626, 669, 672, 712, 771, 776,
        786, 820, 823, 848, 862, 891, 913, 917, 922, 927, 935, 951, 959, 971,
        977, 983, 984, 987,

In [78]:
topico = 0
for doc in docs_per_topics_7[topico]:
    print(doc," ",data_title[doc], probs_7[doc])
len(docs_per_topics_7[topico])

0   Elecciones en Venezuela: María Corina Machado pidió más apoyo de Noruega para garantizar unas presidenciales libres 0.9094560383444804
2   Día de los veteranos de Malvinas: el impacto y las secuelas del trauma psíquico en los excombatientes 0.9482652355492277
4   El galante gesto de Diego Maradona con Mariana A: "Lo que hizo solo habla de su generosidad" 0.9221780009267218
5   Se acerca el final de Rigo: RCN confirmó la fecha del último capítulo 1.0
8   Chipre asegura que el corredor humanitario a Gaza se mantendrá pese a la muerte de los cooperantes de WCK 0.8878870742709135
12   Trias no descarta un pacto Junts-PSC tras el 12-M: "El problema es para hacer qué" 0.8291469237912014
13   Un niño de 12 años mató a un compañero de clase e hirió a otros dos con un arma de fuego en Finlandia 0.8665561955378658
14   Accidente en la General Paz: un auto terminó incrustado debajo de un camión 0.9079198318994458
17   Una joven de 19 años fue baleada en la puerta de su casa por dos hombres qu

64

In [79]:
topico = 1
for doc in docs_per_topics_7[topico]:
    print(doc," ",data_title[doc], probs_7[doc])
len(docs_per_topics_7[topico])

1   El populismo derivó en que los argentinos financien el crecimiento de los países desarrollados 1.0
3   La revolución del efectivo: la Generación Z redescubre el dinero antiguo para evitar la bancarrota moderna 1.0
6   Dólar versus inflación: ¿cuál debería ser el nivel del tipo de cambio? 1.0
9   Elogios de un profesor argentino que enseña en Harvard: "Para hacer progresismo hay que hacer lo que está haciendo Milei" 0.9459591183206592
11   WhatsApp habilitó más funciones nuevas en abril, conoce la lista 0.9962922144718224
15   Bitcoin: cómo se ha movido en el mercado este 2 de abril 1.0
16   Google dice adiós a una de sus mejores aplicaciones después de seis años de vida 0.9494901968545474
20   Un teclado puede albergar más gérmenes que un inodoro: qué tipo de bacterías 1.0
30   Criptomonedas: el precio de ethereum para este día 0.9430431696779146
35   Se disparó el interés por alquilar departamentos de 4 ambientes en CABA: la oferta subió más de 80 por ciento 1.0
40   Criptomonedas

19

In [29]:
len(docs_per_topics_7[-1])

234

- 7 Con 100 registros:
    - sin preprocesamiento 
    - con vocabulario + repeticion de palabras: mas de dos veces
    - sin ajustes de bertopic
    - con TFIDFVectorizer que aplica al vocabulario + min_df y max_df
        - Resultado: Clasifica en dos topicos tambien, se obtiene un resultado similar al Test 3
            - del topico 0, de 65 (Test 3) a 64.
            - del topico 1, de 18 (Test 3) a 19.

Cabe mencionar que la cantidad de palabras del vocabulario es de 259 vs 2279 que se obtienen de TFIDF sobre el total de los documentos

## Test 8- Cfg Test 6 + configuracion en max_df y min_df + preprocesado de docs

In [52]:
# Limpiar texto
cleaner = Cleaning_text()

docs_clean = []
for doc in data:
    clean_txt = cleaner.unicode(doc)
    clean_txt = cleaner.urls(clean_txt)
    clean_txt = cleaner.simbols(clean_txt)
    clean_txt = cleaner.accents_emojis(clean_txt)
    clean_txt = cleaner.escape_sequence(clean_txt)
    docs_clean.append(cleaner.str_lower(clean_txt))

# Eliminamos stopwords del corpus
preprocessed_data = []
registro = ""
for txt in docs_clean:
    for word in txt.split():
        if word.isalpha() and not word in SPANISH_STOPWORDS_SIN_ACENTOS:
            registro += word + " "
        elif word.isnumeric():
            registro += word + " "
    preprocessed_data.append(registro+' ')
    registro = ""

In [53]:
tfidf_vectorizer_8 = TfidfVectorizer(
    # tokenizer=tokenizer,
    max_df=0.9,
    min_df=0.05,
    ngram_range=(1, 2),
    stop_words=SPANISH_STOPWORDS,
    lowercase=True,
    #vocabulary=vocab_entities,
    max_features=100_000
)
tfidf_vectorizer_8.fit(preprocessed_data)

topic_model_8 = BERTopic(
    language='spanish',
    #calculate_probabilities=True,
    vectorizer_model=tfidf_vectorizer_8
)

topics_8, probs_8 = topic_model_8.fit_transform(preprocessed_data)

In [55]:
T_8 = topic_model_8.get_document_info(data)
docs_per_topics_8 = T_8.groupby(["Topic"]).apply(lambda x: x.index).to_dict()
docs_per_topics_8

{0: Index([  0,   1,   2,   3,   4,   5,   6,   7,   9,  10,
        ...
        990, 991, 992, 993, 994, 995, 996, 997, 998, 999],
       dtype='int64', length=954),
 1: Index([  8,  52,  67,  72, 123, 151, 182, 201, 226, 260, 288, 292, 340, 464,
        532, 603, 654, 911, 918, 928, 933, 957, 978],
       dtype='int64'),
 2: Index([191, 246, 279, 303, 312, 345, 372, 434, 521, 529, 649, 692, 713, 727,
        728, 761, 809, 822, 854, 876, 896, 931, 980],
       dtype='int64')}

In [38]:
topico = 0
for doc in docs_per_topics_8[topico]:
    print(doc," ",data_title[doc], probs_8[doc])
len(docs_per_topics_8[topico])

0   Elecciones en Venezuela: María Corina Machado pidió más apoyo de Noruega para garantizar unas presidenciales libres 1.0
1   El populismo derivó en que los argentinos financien el crecimiento de los países desarrollados 1.0
2   Día de los veteranos de Malvinas: el impacto y las secuelas del trauma psíquico en los excombatientes 1.0
3   La revolución del efectivo: la Generación Z redescubre el dinero antiguo para evitar la bancarrota moderna 1.0
4   El galante gesto de Diego Maradona con Mariana A: "Lo que hizo solo habla de su generosidad" 1.0
5   Se acerca el final de Rigo: RCN confirmó la fecha del último capítulo 1.0
6   Dólar versus inflación: ¿cuál debería ser el nivel del tipo de cambio? 1.0
7   La lectura en voz alta: clave para el desarrollo infantil en el hogar y la escuela 1.0
9   Elogios de un profesor argentino que enseña en Harvard: "Para hacer progresismo hay que hacer lo que está haciendo Milei" 1.0
10   En abril crecerá la protesta sindical: se prevé una oleada de pa

952

In [39]:
topico = 1
for doc in docs_per_topics_8[topico]:
    print(doc," ",data_title[doc], probs_8[doc])
len(docs_per_topics_8[topico])

8   Chipre asegura que el corredor humanitario a Gaza se mantendrá pese a la muerte de los cooperantes de WCK 0.45244457826985807
52   El régimen de Irán amenazó nuevamente a Israel tras el bombardeo al consulado en Siria: "No quedará sin respuesta" 0.5159045422800516
67   Arabia Saudí condena el ataque contra el Consulado iraní en Damasco sin mencionar a Israel 1.0
72   Israel culpó a Irán de un ataque con drone que impactó contra un edificio de una base naval en Eilat 0.7833237145811123
123   Cómo es la fábrica de drones Shahed donde Ucrania hizo su ataque más profundo en territorio ruso 1.0
151   El Ejército israelí lamentó la muerte de siete integrantes de una ONG en Gaza y prometió realizar una investigación independiente 0.601443946089523
166   Nueva amenaza de Kim Jong-un: Corea del Norte lanzó otro misil balístico al mar 0.21836616205922452
182   Un terrorista acuchilló e hirió gravemente a un niño y a dos jóvenes 0.4865651054538667
201   Israel bombardeó el consulado de Irán e

25

- 8 Con 1000 registros:
    - con preprocesamiento de docs y stop-words
    - con vocabulario + repeticion de palabras: mas de una vez
    - sin ajustes de bertopic
    - con TFIDFVectorizer que aplica al vocabulario + min_df y max_df
        - Resultado: Clasifica en dos topicos tambien, se obtiene un resultado similar al Test 3
            - del topico 0, de 50 (Test 5) a 48.
            - del topico 1, de 16 (Test 5) a 18.

Cabe mencionar que la cantidad de palabras del vocabulario es de 259 vs 2279 que se obtienen de TFIDF sobre el total de los documentos

### Resumen

|       | Test1 | Test2 | Test3 | Test4 | Test5 | Test6 | Test7 | Test8 |
|-------|-------|-------|-------|-------|-------|-------|-------|-------|
| Topico 0   |  71  |   61 |   65 |   69 |   50 |   64 |   64 |   48 |
| Topico 1   |  21  |   19 |   18 |   21 |   16 |   21 |   19 |   18 |
|  Topico -1 |  8   |   20 |   17 |   10 |   34 |   15 |   17 |   34 |
|  Vocab |     |   2279 |    |    |   543|   259 |   259|   259 |
|  Tiempo  seg  | 18.5 | 21.7 | 23.6 | 33.2 | 27.1 | 19.7 | 26.1 | 25.7 |



## Test 9- Test 5 (solo con palabras del vocabulario que se repiten mas de una vez) + cfg BERTopic

In [7]:
# Cargar el modelo de spaCy para español
spa = spacy.load("es_core_news_lg")

# Stopwords
SPANISH_STOPWORDS = list(pd.read_csv(PATH+'spanish_stop_words.csv' )['stopwords'].values)

# Detectar entidades para todos los documentos usando spaCy
entities_spa = []
for doc in tqdm(data):
    extract = spa(doc)
    entities_spa.append([(ent.text, ent.label_) for ent in extract.ents])

# Procesamiento de entidades encontradas
entities = []
word_count = {}
for item in tqdm(entities_spa):
    for ent in item:
        if ent[1] == 'PER' or ent[1] == 'ORG' or ent[1] == 'LOC':
            token = str(ent[0]).lower()  
            if token not in word_count:
                word_count[token] = {'count': 0, 'original': ent[0]}
                
            word_count[token]['count'] += 1   
    
    # Ordenar el diccionario por el valor del conteo en orden descendente
    sorted_word_count = dict(sorted(word_count.items(), key=lambda item: item[1]['count'], reverse=True))  

    word_count = {}

    # Crear la lista de entidades procesadas por noticia (entrenamiento)
    pre_entities = clean_all( [key for key, _ in sorted_word_count.items()
                                if key not in SPANISH_STOPWORDS and len(key.split()) < 5 and _['count'] > 1 ] )

    # entidades para entrenar
    entities.append( pre_entities )

100%|██████████| 1000/1000 [00:00<00:00, 2930.26it/s]


In [12]:
sorted_word_count

{'policía': {'count': 5, 'original': 'Policía'},
 'vantaa': {'count': 2, 'original': 'Vantaa'},
 'viertola': {'count': 2, 'original': 'Viertola'},
 'reuters': {'count': 2, 'original': 'Reuters'},
 'finlandia': {'count': 1, 'original': 'Finlandia'},
 'aeropuerto de helsinki-vantaa': {'count': 1,
  'original': 'aeropuerto de Helsinki-Vantaa'},
 'newsletters de la nueva': {'count': 1,
  'original': 'Newsletters de La Nueva'},
 'interior': {'count': 1, 'original': 'Interior'},
 'mari rantanen': {'count': 1, 'original': 'Mari Rantanen'},
 'siltamaki': {'count': 1, 'original': 'Siltamaki'},
 'rato': {'count': 1, 'original': 'Rato'}}

In [142]:
# Vocabulario
preprocessed_entities = []
registro = ""
for array in entities:
    for ent in array:
        if len(ent.split()) > 2:
            for word in ent.split():
                if word.isalpha() and len(word) > 2 and len(word) < 15 and word not in SPANISH_STOPWORDS:    
                    preprocessed_entities.append(word)
                elif word.isnumeric() and len(word) == 4:
                    preprocessed_entities.append(word)
        else:
            for word in ent.split():
                if word.isalpha() and len(word) > 2 and len(word) < 15 and word not in SPANISH_STOPWORDS:  
                    registro += word + " "
            if registro:
                preprocessed_entities.append(registro.strip())
                registro = ""

vocab_entities = list(set([item for item in preprocessed_entities]))
len(vocab_entities)

2914

In [143]:
tfidf_vectorizer_9 = TfidfVectorizer(
    # tokenizer=tokenizer,
    max_df=0.9,
    min_df=0.1,
    ngram_range=(1, 2),
    #stop_words=SPANISH_STOPWORDS,
    lowercase=True,
    vocabulary=vocab_entities,
    # max_features=100_000
)
tfidf_vectorizer_9.fit(data)

# Step 1 - Extract embeddings
embedding_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
# Step 2 - Reduce dimensionality
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine')
# Step 3 - Cluster reduced embeddings
hdbscan_model = HDBSCAN(min_cluster_size=10, metric='euclidean', cluster_selection_method='eom', prediction_data=True)
# Step 4 - Tokenize topics
vectorizer_model = tfidf_vectorizer_9
# Step 5 - Create topic representation
ctfidf_model = ClassTfidfTransformer()
# Step 6 - (Optional) Fine-tune topic representations with a `bertopic.representation` model
representation_model = KeyBERTInspired()

# All steps together
topic_model_9 = BERTopic(
  embedding_model=embedding_model,            # Step 1 - Extract embeddings
  umap_model=umap_model,                      # Step 2 - Reduce dimensionality
  hdbscan_model=hdbscan_model,                # Step 3 - Cluster reduced embeddings
  vectorizer_model=tfidf_vectorizer_9,          # Step 4 - Tokenize topics
  ctfidf_model=ctfidf_model,                  # Step 5 - Extract topic words
  representation_model=representation_model,  # Step 6 - (Optional) Fine-tune topic represenations
  language='spanish',
  #calculate_probabilities=True
)

topics_9, probs_9 = topic_model_9.fit_transform(data)

In [144]:
T_9 = topic_model_9.get_document_info(data)
docs_per_topics_9 = T_9.groupby(["Topic"]).apply(lambda x: x.index).to_dict()
docs_per_topics_9

{-1: Index([  3,  10,  12,  24,  25,  26,  32,  34,  43,  46,
        ...
        945, 946, 948, 962, 964, 968, 974, 982, 993, 998],
       dtype='int64', length=210),
 0: Index([ 23,  39,  82,  93,  95,  99, 101, 105, 106, 119,
        ...
        952, 958, 963, 970, 975, 976, 979, 981, 988, 990],
       dtype='int64', length=162),
 1: Index([  4,   5,  18,  33,  36,  41,  44,  58,  60,  63,
        ...
        935, 951, 959, 971, 972, 977, 983, 984, 987, 989],
       dtype='int64', length=129),
 2: Index([ 13,  14,  17,  29,  49,  50,  53,  57,  89,  91,
        ...
        941, 943, 949, 953, 955, 960, 967, 973, 986, 999],
       dtype='int64', length=119),
 3: Index([  1,   6,   9,  35, 104, 114, 168, 172, 180, 192, 193, 205, 212, 213,
        268, 276, 316, 326, 338, 342, 353, 380, 388, 397, 436, 452, 461, 463,
        466, 483, 484, 499, 518, 540, 562, 566, 587, 594, 604, 619, 628, 638,
        646, 659, 663, 671, 678, 679, 681, 699, 708, 757, 758, 764, 773, 793,
        818, 828

In [145]:
topico = 0
for doc in docs_per_topics_9[topico]:
    print(doc," ",data_title[doc], probs_9[doc])
len(docs_per_topics_9[topico])

23   Arranca la Copa Sudamericana: la agenda con el debut de Belgrano de Córdoba, Argentinos Juniors y Defensa y Justicia 1.0
39   La sugerente frase de una ex figura de Boca Juniors antes de enfrentar al Inter Miami: "Messi es un atractivo diferente para los árbitros" 1.0
82   Arden las tablas de la Copa de la Liga: Godoy Cruz es el primer clasificado, Boca afuera, River adentro y los posibles cruces de cuartos 0.8298092386529479
93   Comienza la ilusión continental: premios, formatos, partidos y todo lo que hay que saber de las Copas Libertadores y Sudamericana 1.0
95   La portada de un diario español que profundiza sobre el interés del Real Madrid por Franco Mastantuono: "Lo quieren ya" 0.8751114774479423
99   Vivió y corrió en Inglaterra durante la Guerra de Malvinas, compitió contra Ayrton Senna y probó un Fórmula 1: la increíble historia de Enrique Mansilla 0.6907130632257792
101   ¿En qué consiste el 'Meet & Greet Upgrade' con Travis Barker y cuánto cuesta? 0.45435059034392167
1

162

In [146]:
topico = 10
for doc in docs_per_topics_9[topico]:
    print(doc," ",data_title[doc], probs_9[doc])
len(docs_per_topics_9[topico])

31   Murió la escritora Maryse Condé, activista contra la esclavitud y el racismo que ganó el "Nobel alternativo" 0.44481632422857215
54   Cinco libros esenciales para conocer la obra de Maryse Condé 0.7236069956502477
110   El desgarrador mensaje de la madre de la esposa del golfista Emilio Domínguez que murió de dengue 1.0
247   Quién era María Victoria De La Mota Claverie, la esposa del golfista Emilio Domínguez que murió por dengue 1.0
267   Jaime López Rivarola sobre el fallecimiento de la esposa de Emilio Domínguez: "Queremos que sepa que todos sus amigos están pensando en él" 0.5327425952750301
296   Quién era María Victoria De La Mota Claverie, la mujer del golfista argentino que murió por dengue 1.0
297   Quién era María Victoria De La Mota Claverie, la joven esposa del "Puma" Domínguez que murió por dengue 0.8884201496296111
301   Murió de dengue la mujer de Emilio "Puma" Domínguez, reconocido jugador de golf argentino 0.9863040097255273
363   Murió de dengue la esposa del go

20

In [104]:
len(docs_per_topics_9[-1])

215

In [105]:
# Obtener las palabras clave de un tema específico
topic_keywords = topic_model_9.get_topic(10)  # Cambiar 0 por el ID del tema deseado
print(topic_keywords)


[('karina milei', 0.5174929), ('monumento', 0.45645806), ('jorge macri', 0.42519456), ('secretaria', 0.41240472), ('argentinos', 0.40678596), ('milei', 0.4064965), ('presidente', 0.40557232), ('reino', 0.4045558), ('carlos sadir', 0.40445343), ('senado', 0.402051)]


- 9. Con 1000 registros:
    - Es muy importante validar el tipo de sentence transformer, en este caso debe ser multilingual

## Test 10- Documentos preprocesados + cfg BERTopic

In [98]:
# Stopwords
SPANISH_STOPWORDS = list(pd.read_csv(PATH+'spanish_stop_words.csv' )['stopwords'].values)
SPANISH_STOPWORDS_SIN_ACENTOS = [unicodedata.normalize('NFKD', word).encode('ASCII', 'ignore').decode('ASCII') for word in SPANISH_STOPWORDS] 


In [108]:
# Limpiar texto
cleaner = Cleaning_text()

docs_clean = []
for doc in data:
    clean_txt = cleaner.unicode(doc)
    clean_txt = cleaner.urls(clean_txt)
    clean_txt = cleaner.simbols(clean_txt)
    clean_txt = cleaner.accents_emojis(clean_txt)
    clean_txt = cleaner.escape_sequence(clean_txt)
    docs_clean.append(cleaner.str_lower(clean_txt))

# Eliminamos stopwords del corpus
preprocessed_data = []
registro = ""
for txt in docs_clean:
    for word in txt.split():
        if word.isalpha() and not word in SPANISH_STOPWORDS_SIN_ACENTOS:
            registro += word + " "
        elif word.isnumeric():
            registro += word + " "
    preprocessed_data.append(registro+' ')
    registro = ""

In [110]:
tfidf_vectorizer_10 = TfidfVectorizer(
    # tokenizer=tokenizer,
    max_df=0.9,
    min_df=0.1,
    ngram_range=(1, 2),
    #stop_words=SPANISH_STOPWORDS,
    lowercase=True,
    #vocabulary=vocab_entities,
    # max_features=100_000
)
tfidf_vectorizer_10.fit(preprocessed_data)

# Step 1 - Extract embeddings
embedding_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
# Step 2 - Reduce dimensionality
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine')
# Step 3 - Cluster reduced embeddings
hdbscan_model = HDBSCAN(min_cluster_size=10, metric='euclidean', cluster_selection_method='eom', prediction_data=True)
# Step 4 - Tokenize topics
vectorizer_model = tfidf_vectorizer_10
# Step 5 - Create topic representation
ctfidf_model = ClassTfidfTransformer()
# Step 6 - (Optional) Fine-tune topic representations with a `bertopic.representation` model
representation_model = KeyBERTInspired()

# All steps together
topic_model_10 = BERTopic(
  embedding_model=embedding_model,            # Step 1 - Extract embeddings
  umap_model=umap_model,                      # Step 2 - Reduce dimensionality
  hdbscan_model=hdbscan_model,                # Step 3 - Cluster reduced embeddings
  vectorizer_model=tfidf_vectorizer_10,          # Step 4 - Tokenize topics
  ctfidf_model=ctfidf_model,                  # Step 5 - Extract topic words
  representation_model=representation_model,  # Step 6 - (Optional) Fine-tune topic represenations
  language='spanish',
  #calculate_probabilities=True
)

topics_10, probs_10 = topic_model_10.fit_transform(preprocessed_data)

In [111]:
T_10 = topic_model_10.get_document_info(data)
docs_per_topics_10 = T_10.groupby(["Topic"]).apply(lambda x: x.index).to_dict()
docs_per_topics_10

{-1: Index([  2,  15,  19,  30,  32,  40,  46,  53,  59,  62,
        ...
        945, 946, 948, 961, 962, 964, 972, 982, 985, 993],
       dtype='int64', length=155),
 0: Index([  3,   4,   5,   7,  11,  12,  13,  14,  16,  17,
        ...
        979, 981, 983, 984, 986, 987, 988, 989, 990, 999],
       dtype='int64', length=500),
 1: Index([ 10,  24,  25,  29,  35,  45,  56,  65,  73,  80,  84, 100, 108, 111,
        124, 129, 131, 132, 157, 160, 161, 169, 202, 205, 206, 207, 237, 252,
        255, 293, 300, 376, 398, 402, 407, 410, 421, 428, 442, 456, 473, 493,
        514, 520, 531, 541, 556, 557, 571, 580, 605, 642, 647, 658, 665, 673,
        674, 703, 718, 720, 723, 737, 740, 767, 790, 803, 804, 810, 825, 834,
        842, 853, 858, 870, 873, 919, 926, 942, 992, 995],
       dtype='int64'),
 2: Index([  1,   6,   9,  34,  51,  61, 112, 165, 168, 172, 180, 192, 193, 213,
        233, 240, 243, 276, 280, 316, 326, 335, 338, 342, 346, 353, 380, 397,
        436, 452, 483, 484, 499

In [112]:
topico = 0
for doc in docs_per_topics_9[topico]:
    print(doc," ",data_title[doc], probs_9[doc])
len(docs_per_topics_9[topico])

23   Arranca la Copa Sudamericana: la agenda con el debut de Belgrano de Córdoba, Argentinos Juniors y Defensa y Justicia 1.0
39   La sugerente frase de una ex figura de Boca Juniors antes de enfrentar al Inter Miami: "Messi es un atractivo diferente para los árbitros" 1.0
82   Arden las tablas de la Copa de la Liga: Godoy Cruz es el primer clasificado, Boca afuera, River adentro y los posibles cruces de cuartos 0.778884716776962
93   Comienza la ilusión continental: premios, formatos, partidos y todo lo que hay que saber de las Copas Libertadores y Sudamericana 1.0
95   La portada de un diario español que profundiza sobre el interés del Real Madrid por Franco Mastantuono: "Lo quieren ya" 0.8284041338475985
99   Vivió y corrió en Inglaterra durante la Guerra de Malvinas, compitió contra Ayrton Senna y probó un Fórmula 1: la increíble historia de Enrique Mansilla 0.6326343119050765
101   ¿En qué consiste el 'Meet & Greet Upgrade' con Travis Barker y cuánto cuesta? 0.5379467077628721
105

166

In [113]:
topico = 10
for doc in docs_per_topics_9[topico]:
    print(doc," ",data_title[doc], probs_9[doc])
len(docs_per_topics_9[topico])

19   En el homenaje a los caídos en Malvinas, Milei convocó a una nueva era de reconciliación con las Fuerzas Armadas 0.5402068514750463
27   Milei y Villarruel encabezarán el acto central por el 42º aniversario de la Guerra de Malvinas 1.0
38   La emoción de la vicepresidenta al recordar a su padre: quién fue el héroe de Malvinas Eduardo Marcelo Villarruel 0.849109169792249
103   Milei y Villarruel encabezarán el acto central a 42 años de Malvinas: habrá cadena nacional 1.0
132   Alberto Fernández y Cristina Kirchner se expresaron por el Día del Veterano y los Caídos en Malvinas 0.6844668110411296
171   Milei espera que se mantenga el descenso de la inflación en los próximos meses 0.5427556446918097
228   Milei rendirá homenaje a los caídos en Malvinas, a 42 años del inicio de la Guerra 1.0
245   Vendimia para Todxs cerró su edición 2024 con una fiesta que reunió a 1.500 personas 0.5543662670901087
258   El Gobernador homenajeo a los Veteranos de Malvinas 1.0
311   Se cumplen 19 años 

22

In [114]:
len(docs_per_topics_9[-1])

215

In [115]:
# Obtener las palabras clave de un tema específico
topic_keywords = topic_model_9.get_topic(10)  # Cambiar 0 por el ID del tema deseado
print(topic_keywords)


[('karina milei', 0.5174929), ('monumento', 0.45645806), ('jorge macri', 0.42519456), ('secretaria', 0.41240472), ('argentinos', 0.40678596), ('milei', 0.4064965), ('presidente', 0.40557232), ('reino', 0.4045558), ('carlos sadir', 0.40445343), ('senado', 0.402051)]


- 9. Con 1000 registros:
    - Es muy importante validar el tipo de sentence transformer, en este caso debe ser multilingual

# Evaluacion final

In [137]:
topico = 15
for doc in docs_per_topics_5[topico][:20]:
    print(doc," ",data_title[doc], probs_5[doc])
len(docs_per_topics_5[topico])

264   La empresa propietaria de la Fórmula 1 compra Moto GP 1.0
333   Los dueños de la Fórmula 1 pagaron miles de millones para subirse a MotoGP 1.0
515   Legislatura de la provincia | Últimas noticias El Tribuno 1.0
648   Día Mundial del autismo | Últimas noticias El Tribuno 1.0
688   Yerba Buena: llegan los radares a la avenida Perón, ¿a qué velocidad podés circular? 1.0
724   DALE AL MEDIO                  - TyC Sports 1.0
747   entrevista exclusiva El Tribuno | Últimas noticias El Tribuno 1.0
843   La Fórmula 1 compra MotoGP: los detalles del acuerdo multimillonario - TyC Sports 1.0
884   Turismo en Semana Santa | Últimas noticias El Tribuno 1.0
920   Liga Argentina de Básquetbol | Últimas noticias El Tribuno 1.0
985   Día de los Veteranos y Caídos en Malvinas | Últimas noticias El Tribuno 1.0


11

In [138]:
topico = 15
for doc in docs_per_topics_9[topico][:20]:
    print(doc," ",data_title[doc], probs_9[doc])
len(docs_per_topics_9[topico])

136   Marzo fue el décimo mes consecutivo en romper récords de altas temperaturas globales 1.0
162   Último día del fin de semana largo: cuál es el pronóstico del tiempo para este martes 1.0
223   Emiten alerta naranja por viento y lluvia para este martes 1.0
254   Pronóstico: para mañana se espera un aumento de la temperatura y alerta por Zonda y sectores del Gan Mendoza 1.0
354   El tiempo en Santiago del Estero: máxima de 26ºC y probabilidad de precipitaciones para el inicio de abril 1.0
359   Inicio de semana fresco y con un cielo parcialmente nublado - MDZ Online 1.0
420   "Villa Mitre será tan duro como lo fue River el viernes", dijo el DT de Huracán 0.47656154426899694
505   El cielo estará nublado y se esperan lluvias durante toda la jornada en Tucumán 1.0
517   Lunes caluroso en Misiones, con pronostico de lluvias y chaparrones aislados 1.0
907   Clima en Córdoba: fuerte descenso de la temperatura para el lunes, ¿llega la lluvia? 1.0
966   Alerta por viento y lluvia, este mart

12

In [42]:
titulos = [
    "Javier Milei",
    "Guillermo Francos",
    "Milei",
    "Javier",
    "Diana Mondino"
]



In [43]:
# Obtener las últimas palabras de las entidades con más de una palabra
ultimas_palabras = [titulo.split()[-1] for titulo in titulos if len(titulo.split()) > 1]
ultimas_palabras

['Milei', 'Francos', 'Mondino']

In [46]:
# Filtrar si las últimas palabras coinciden con alguna unica palabra
filtro_ulp = [titulo for titulo in titulos if not (len(titulo.split()) == 1 and titulo in ultimas_palabras)]
filtro_ulp

['Javier Milei', 'Guillermo Francos', 'Javier', 'Diana Mondino']

In [45]:
# Obtener las palabras únicas
unicas_palabras = [ palabra for palabra in resultado if len(palabra.split()) == 1]
unicas_palabras


['Javier']

In [47]:
# Filtrar si las palabras únicas coinciden con las una entidad con más de una palabra
filtro_unp = [ titulo for titulo in filtro_ulp if not titulo in unicas_palabras ]
filtro_unp

['Javier Milei', 'Guillermo Francos', 'Diana Mondino']